### 스타벅스 서울 매장위치 크롤링

#### 사용 모듈
- 셀레니움
- 뷰티플수프
- 판다스
- 포리움

In [1]:
!pip install beautifulsoup4


[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

##### 웹드라이버 자동선택

- html 소스를 확인 - 값이 동적으로 변경되는 부분 
- CSS_SELECTOR 구문 검색 방법
    (# 아이디(페이지상에 유일함), . 클래스(여러개)))
    ex) #container > div > form > fieldset > div > section > article.find_store_cont > article > article.nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a

In [5]:
btn_seoul_link = '#container > div > form > fieldset > div > section > article.find_store_cont > ' + \
            ' article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, btn_seoul_link).click()

In [6]:
btn_all = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, btn_all).click()

##### BeautifulSoup로 html 크롤링


In [7]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [8]:
len(soup.select('li.quickResultLstCon'))

601

In [10]:
sb_seoul_list = soup.select('li.quickResultLstCon')
sb_seoul_list

[<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>,
 <li class="quickResultLstCon" data-code="3672" data-hlytag="null" data-index="1" data-lat="37.510178" data-long="127.022223" data-name="논현역사거리" data-storecd="1434" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="논현역사거리" data-store="1434" data-yn="N">논현역사거리  </strong> <p class="result_details">서울특별시 강남구 강남대로 538 (논현동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>,
 <li class="quickResultLstCon" data-code="3858" data-hlytag="null" data-index="2" data-lat="37.5139309" data-long="127.0206057" data-name="신사역성일빌딩" data-storecd="1595" style="bac

In [11]:
sb_seoul_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [12]:
# 매장정보
sb_store = sb_seoul_list[0]
name = sb_store.select('strong')[0].text.strip() # 매장이름
lat = sb_store['data-lat'] # 매장위치 위도값
lng = sb_store['data-long'] # 경도값
address = sb_store.select('p')[0].get_text(separator='|').split('|')[0]
tel = sb_store.select('p')[0].get_text(separator='|').split('|')[1]
type = sb_store.select('i')[0]['class'][0].split('_')[1]

print(name)
print(lat, lng,)
print(type, address, tel, sep=' / ')

역삼아레나빌딩
37.501087 127.043069
general / 서울특별시 강남구 언주로 425 (역삼동) / 1522-3232


In [13]:
sb_list = []

for item in sb_seoul_list:
    name = item.select('strong')[0].text.strip() # 매장이름
    lat = item['data-lat'] # 매장위치 위도값
    lng = item['data-long'] # 경도값
    address = item.select('p')[0].get_text(separator='|').split('|')[0]
    tel = item.select('p')[0].get_text(separator='|').split('|')[1]
    type = item.select('i')[0]['class'][0].split('_')[1]

    sb_list.append([name, lat, lng, address, tel, type])

##### tqdm 

프로그레스바로 진행사항을 표시해주는 모듈

```
!pip install tqdm
```

In [14]:
!pip install tqdm


[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from tqdm.autonotebook import tqdm

C:\Users\user\AppData\Local\Temp\ipykernel_4464\987820437.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [16]:
sb_list = []

for item in tqdm(sb_seoul_list):
    name = item.select('strong')[0].text.strip() # 매장이름
    lat = item['data-lat'] # 매장위치 위도값
    lng = item['data-long'] # 경도값
    address = item.select('p')[0].get_text(separator='|').split('|')[0]
    tel = item.select('p')[0].get_text(separator='|').split('|')[1]
    type = item.select('i')[0]['class'][0].split('_')[1]

    sb_list.append([name, lat, lng, address, tel, type])

  0%|          | 0/601 [00:00<?, ?it/s]

100%|██████████| 601/601 [00:00<00:00, 3478.65it/s]


In [17]:
len(sb_list)

601

In [18]:
# DataFrame에 저장
columns = ['매장명', '위도', '경도', '주소', '전화번호', '매장타입']
df_sb_seoul = pd.DataFrame(data=sb_list, columns=columns)

In [19]:
df_sb_seoul

,매장명,위도,경도,주소,전화번호,매장타입
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,general
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,general
2,신사역성일빌딩,37.5139309,127.0206057,서울특별시 강남구 강남대로 584 (논현동),1522-3232,general
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,general
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,reserve
...,...,...,...,...,...,...
596,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232,general
597,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,general
598,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,general
599,양원역,37.6066536267232,127.106359790053,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,general


In [20]:
df_sb_seoul.head()

,매장명,위도,경도,주소,전화번호,매장타입
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,general
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,general
2,신사역성일빌딩,37.5139309,127.0206057,서울특별시 강남구 강남대로 584 (논현동),1522-3232,general
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,general
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,reserve


In [21]:
df_sb_seoul.tail()

,매장명,위도,경도,주소,전화번호,매장타입
596,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232,general
597,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,general
598,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,general
599,양원역,37.6066536267232,127.106359790053,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,general
600,중화역,37.60170912407773,127.07841136432036,서울특별시 중랑구 봉화산로 35,1522-3232,general


In [22]:
df_sb_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     601 non-null    object
 1   위도      601 non-null    object
 2   경도      601 non-null    object
 3   주소      601 non-null    object
 4   전화번호    601 non-null    object
 5   매장타입    601 non-null    object
dtypes: object(6)
memory usage: 28.3+ KB


In [23]:
df_sb_seoul.to_excel('./starbucks_seoul.xlsx', index=False)

#### 전처리 및 시각화

- 지도 모듈 설치
    - folium

```python
!pip install folium
```

In [24]:
!pip install folium


[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
df_sb_seoul

,매장명,위도,경도,주소,전화번호,매장타입
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,general
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,general
2,신사역성일빌딩,37.5139309,127.0206057,서울특별시 강남구 강남대로 584 (논현동),1522-3232,general
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,general
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,reserve
...,...,...,...,...,...,...
596,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232,general
597,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,general
598,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,general
599,양원역,37.6066536267232,127.106359790053,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,general


In [26]:
# 간단한 전처리 '구' 컬럼 추가작업
df_sb_seoul['주소'][600].split()[1]

'중랑구'

In [27]:
gu_list = []

for item in df_sb_seoul['주소']:
    gu = item.split()[1]
    gu_list.append(gu)

In [28]:
df_sb_seoul['구'] = gu_list

In [29]:
df_sb_seoul

,매장명,위도,경도,주소,전화번호,매장타입,구
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,general,강남구
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,general,강남구
2,신사역성일빌딩,37.5139309,127.0206057,서울특별시 강남구 강남대로 584 (논현동),1522-3232,general,강남구
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,general,강남구
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,reserve,강남구
...,...,...,...,...,...,...,...
596,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232,general,중랑구
597,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,general,중랑구
598,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,general,중랑구
599,양원역,37.6066536267232,127.106359790053,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,general,중랑구


#### 대한민국 행정구역 위경도 전처리

In [30]:
df_loc = pd.read_excel('./대한민국_행정구역_위경도.xlsx')
df_loc

,docity,do,city,longitude,latitude
0,강원강릉시,강원,강릉시,128.878497,37.749136
1,강원고성군,강원,고성군,128.470164,38.377961
2,강원동해시,강원,동해시,129.116633,37.521931
3,강원삼척시,강원,삼척시,129.167489,37.447086
4,강원속초시,강원,속초시,128.594167,38.204275
...,...,...,...,...,...
290,충청충주시,충청,충주시,127.928144,36.988181
291,충청태안군,충청,태안군,126.299975,36.742667
292,충청한누리대로,충청,한누리대로,127.289926,36.485450
293,충청홍성군,충청,홍성군,126.662908,36.598361


In [31]:
df_loc_seoul = df_loc[df_loc['do'] == '서울']
df_loc_seoul.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 138 to 162
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   docity     25 non-null     object 
 1   do         25 non-null     object 
 2   city       25 non-null     object 
 3   longitude  25 non-null     float64
 4   latitude   25 non-null     float64
dtypes: float64(2), object(3)
memory usage: 1.2+ KB


In [32]:
df_gu_count = df_sb_seoul.pivot_table(index='구', values='매장명', aggfunc='count').rename(columns={'매장명':'스타벅스 매장수'})
df_gu_count.tail()

,스타벅스 매장수
구,
용산구,25
은평구,13
종로구,40
중구,53
중랑구,8


In [33]:
df_loc_seoul.rename(columns={'do':'도', 'city':'구', 'latitude':'위도', 'longitude':'경도'}, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_4464\1184758135.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc_seoul.rename(columns={'do':'도', 'city':'구', 'latitude':'위도', 'longitude':'경도'}, inplace=True)


In [34]:
df_loc_seoul.reset_index(drop=True, inplace=True)
df_loc_seoul.tail()

,docity,도,구,경도,위도
20,서울용산구,서울,용산구,126.967522,37.536094
21,서울은평구,서울,은평구,126.931242,37.599969
22,서울종로구,서울,종로구,126.981642,37.570378
23,서울중구,서울,중구,126.999642,37.561003
24,서울중랑구,서울,중랑구,127.094778,37.603806


In [35]:
df_loc_seoul = df_loc_seoul[['도', '구', '위도', '경도']]

In [36]:
df_loc_seoul.tail()

,도,구,위도,경도
20,서울,용산구,37.536094,126.967522
21,서울,은평구,37.599969,126.931242
22,서울,종로구,37.570378,126.981642
23,서울,중구,37.561003,126.999642
24,서울,중랑구,37.603806,127.094778


In [37]:
df_sb_seoul_stat = pd.merge(df_loc_seoul, df_gu_count, how='left', on='구')

In [38]:
df_sb_seoul_stat.tail()

,도,구,위도,경도,스타벅스 매장수
20,서울,용산구,37.536094,126.967522,25
21,서울,은평구,37.599969,126.931242,13
22,서울,종로구,37.570378,126.981642,40
23,서울,중구,37.561003,126.999642,53
24,서울,중랑구,37.603806,127.094778,8


In [39]:
import folium

In [41]:
sb_map = folium.Map(location=[37.550823, 126.989502], tiles='Stamen Terrain', zoom_start=11)

for idx in df_sb_seoul.index:
    lat = df_sb_seoul.loc[idx, '위도']
    lng = df_sb_seoul.loc[idx, '경도']
    name = df_sb_seoul.loc[idx, '매장명']

    folium.CircleMarker(location=[lat, lng], fill=True, fill_color='green', fill_opacity=1, popup=name,
                        color='yellow', weight=1, radius=3).add_to(sb_map)

sb_map